# CAPSTONE PROJECT #1: Deep Demographic Insights from Amazon Customer Data

**Summary:** With this project, I plan to explore the demographics of our current customer base in order to determine better who to target with our marketing and which messages may resonate more soundly with them. To this end, I will use the customizable order reports that Amazon provides to its sellers through its Seller Central portal as a dataset. These reports include data such as purchase items & quantity, name, address, city, state, and zip code. I will also use datasets from data.gov for the demographic data with which I will perform my analyses. Data will be visualized here in ways that make the insights most clear.

## Part 1: Data Wrangling / Cleaning

Link to [Part 2 - Exploratory Data Analysis (EDA)](capstone1-eda.ipynb)
<br>
Link to [Part 3 - Inferential Statistics](capstone1-is.ipynb)
<br>
Link to [Part 4 - Machine Learning](capstone1-ml.ipynb)
<br>

In [47]:
#Imports
import pandas as pd
import glob
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import ticker
import calendar

In [48]:
#Read in data.gov dataset (only selecting columns of interest)
demo_data_RAW = pd.read_csv('16zpallagi.csv', usecols=['zipcode', 'agi_stub', 'N1', 'NUMDEP', 'ELDERLY', 'A00100',
                                                      'mars1', 'MARS2', 'MARS4'])
#  ZIPCODE	5-digit Zip code
#  AGI_STUB	Size of adjusted gross income
    #1 = $1 under $25,000
    #2 = $25,000 under $50,000
    #3 = $50,000 under $75,000
    #4 = $75,000 under $100,000
    #5 = $100,000 under $200,000
    #6 = $200,000 or more
#  N1	Number of returns
#  NUMDEP	Number of dependents
#  ELDERLY	Number of elderly returns [4]
#  A00100	Adjust gross income (AGI) [2] (in Thousands)
#  MARS1	Number of single returns	  Filing status is single
#  MARS2	Number of joint returns	  Filing status is married filing jointly
#  MARS4	Number of head of household returns	  Filing status is head of household


In [49]:
#Create new copy for cleaning/modification
demo_data = demo_data_RAW.copy()

In [50]:
print(demo_data_RAW.head(10)) #Inspect head

   zipcode  agi_stub      N1   mars1   MARS2   MARS4  NUMDEP  ELDERLY  \
0        0         1  815440  477700  105350  221200  491310   150920   
1        0         2  495830  211930  142340  128890  360480   109680   
2        0         3  263390   83420  137870   36340  182880    75570   
3        0         4  167190   29420  124060   10610  130160    51540   
4        0         5  217440   20240  188080    4880  195990    68080   
5        0         6   57240    2970   51130     400   56220    19330   
6    35004         1    1510     990     220     280     660      340   
7    35004         2    1410     660     360     340     900      270   
8    35004         3     950     330     470     140     660      200   
9    35004         4     650     120     470      50     560      130   

     A00100  
0  10610343  
1  17839904  
2  16193076  
3  14513312  
4  29375736  
5  25824639  
6     19351  
7     51976  
8     58988  
9     56302  


In [51]:
print(demo_data_RAW.tail(10)) #Inspect tail

        zipcode  agi_stub    N1  mars1  MARS2  MARS4  NUMDEP  ELDERLY  A00100
179786    83414         3    40     30     20      0      40        0    2834
179787    83414         4     0      0      0      0       0        0       0
179788    83414         5    40      0     30      0      50       30    5470
179789    83414         6    30      0     30      0       0        0   32793
179790    99999         1  2730   1980    410    340     950      730   33206
179791    99999         2  2020   1100    620    260    1010      620   74226
179792    99999         3  1330    570    630    130     730      460   82628
179793    99999         4   910    280    580     60     550      350   78494
179794    99999         5  1270    240   1010     40     860      510  170427
179795    99999         6   390     90    330      0     210      240  250829


In [52]:
#Clean demo data (use copy of dataframe, demo_data)
#1. Remove summary rows (zipcode = 0, totaled AGI for brackets 1-6)
#2. "ZIP codes with less than 100 returns and those identified as a single building or 
    #nonresidential ZIP code were categorized as “other” (99999)." -> Remove zipcode = 99999
    
demo_data = demo_data[(demo_data['zipcode'] != 0) & (demo_data['zipcode'] != 99999)]

#Ensure no NULL or NaN values
assert demo_data.notna().all().all()
assert demo_data.notnull().all().all()

print(demo_data.head())
print(demo_data.tail())

    zipcode  agi_stub    N1  mars1  MARS2  MARS4  NUMDEP  ELDERLY  A00100
6     35004         1  1510    990    220    280     660      340   19351
7     35004         2  1410    660    360    340     900      270   51976
8     35004         3   950    330    470    140     660      200   58988
9     35004         4   650    120    470     50     560      130   56302
10    35004         5   630     50    560     20     610      130   81057
        zipcode  agi_stub  N1  mars1  MARS2  MARS4  NUMDEP  ELDERLY  A00100
179785    83414         2  40      0     30      0      50       50    1532
179786    83414         3  40     30     20      0      40        0    2834
179787    83414         4   0      0      0      0       0        0       0
179788    83414         5  40      0     30      0      50       30    5470
179789    83414         6  30      0     30      0       0        0   32793


In [53]:
#Inspection reveals that some postal codes only contain 4 digits... Investigate extent of missing data
demo_data['zipcode'] = demo_data['zipcode'].astype(str)
demo_data_badzip = demo_data[demo_data['zipcode'].str.len() < 5]
print(demo_data_badzip.head())
print(demo_data_badzip.shape) #13,942 entries with malformed ZIP Codes!

      zipcode  agi_stub    N1  mars1  MARS2  MARS4  NUMDEP  ELDERLY  A00100
22290    6001         1  2170   1930    150     70     160      490   20138
22291    6001         2   980    660    190    100     280      340   36335
22292    6001         3   990    570    310     80     360      380   61690
22293    6001         4   840    350    400     70     460      360   73155
22294    6001         5  2170    380   1640    110    1920      820  311741
(13942, 9)


In [54]:
#Research confirms that for the cities and states with malformed zip codes, the zip codes begin with 0.
#Excel truncates the leading zero. (#e.g. 06001 becomes 6001 in Excel) 
def format_postal_codes(row):
    zipcode = row['zipcode']
    return zipcode[:5].zfill(5) #Only keep 1st 5 digits and add leading zeroes to 4-digit codes

In [55]:
#Format ZIP Codes and confirm all are fixed
demo_data['zipcode'] = demo_data.apply(format_postal_codes, axis=1)

demo_data_badzip = demo_data[demo_data['zipcode'].str.len() < 5]
print(demo_data_badzip.shape)

(0, 9)


In [56]:
#Read in Amazon order data files (only selecting columns of interest)

amzn_files = glob.glob('amzn_*.csv')
frames = []

for amzn_file in amzn_files:
    df = pd.read_csv(amzn_file, 
                     usecols=['amazon-order-id', 'ship-city', 'ship-state', 'ship-postal-code', 
                                         'purchase-date', 'order-status', 'item-price'], 
                     parse_dates=['purchase-date'])
    frames.append(df)
    
amzn_data_RAW = pd.concat(frames)

print(amzn_data_RAW.head())

       amazon-order-id       purchase-date order-status  item-price  \
0  113-9235397-9394658 2017-04-27 16:14:10      Shipped       74.99   
1  112-1718234-0365053 2017-04-25 18:03:59      Shipped       74.99   
2  111-1795457-0212247 2017-04-24 17:07:36      Shipped       74.99   
3  114-9388024-7222668 2017-04-24 03:07:35      Shipped       59.99   
4  112-7199274-2267420 2017-04-24 00:19:53      Shipped       59.99   

        ship-city ship-state ship-postal-code  
0          BOSTON         MA       02118-1089  
1  EAST WENATCHEE         WA       98802-2603  
2    Saint Joseph         MO            64506  
3       LOS ALTOS         CA       94022-1209  
4          DENVER         CO       80231-4510  


In [57]:
#Create new copy for cleaning/modification
amzn_data = amzn_data_RAW.copy()

In [58]:
#Clean amzn data (use copy of dataframe, amzn_data)
#1. Remove order status = cancelled (Cancelled purchases)
#2. Remove item-price == NaN (Internal orders)
#3. Re-format postal codes to 5-digit

amzn_data = amzn_data[(amzn_data['order-status'] != 'Cancelled') & (amzn_data['item-price'].isna() == False)]
amzn_data = amzn_data.rename(columns={'ship-postal-code' : 'zipcode'})

amzn_data['zipcode'] = amzn_data.apply(format_postal_codes, axis=1)

amzn_data_badzip = amzn_data[amzn_data['zipcode'].str.len() < 5]
print(amzn_data_badzip.shape) #0 entries with malformed zip codes

print(amzn_data.head())

(0, 7)
       amazon-order-id       purchase-date order-status  item-price  \
0  113-9235397-9394658 2017-04-27 16:14:10      Shipped       74.99   
1  112-1718234-0365053 2017-04-25 18:03:59      Shipped       74.99   
2  111-1795457-0212247 2017-04-24 17:07:36      Shipped       74.99   
3  114-9388024-7222668 2017-04-24 03:07:35      Shipped       59.99   
4  112-7199274-2267420 2017-04-24 00:19:53      Shipped       59.99   

        ship-city ship-state zipcode  
0          BOSTON         MA   02118  
1  EAST WENATCHEE         WA   98802  
2    Saint Joseph         MO   64506  
3       LOS ALTOS         CA   94022  
4          DENVER         CO   80231  


In [59]:
#Cleaning complete - strip unnecessary columns

amzn_data = amzn_data[['amazon-order-id', 'purchase-date', 'zipcode']]
print(amzn_data.head())

       amazon-order-id       purchase-date zipcode
0  113-9235397-9394658 2017-04-27 16:14:10   02118
1  112-1718234-0365053 2017-04-25 18:03:59   98802
2  111-1795457-0212247 2017-04-24 17:07:36   64506
3  114-9388024-7222668 2017-04-24 03:07:35   94022
4  112-7199274-2267420 2017-04-24 00:19:53   80231
